# Simple Input Processing

This is a small collection of "proper implementations" of basic incoming data transformations for basic types. As data is often coming in over the web in a textual form, these primarily target string transformations.

## Boolean Values

There are many "natural" ways to express if a value is on or true, false or off, and so forth. Over HTTP, type information is lost and all values are transferred as strings, your checkboxes will need an acceptable value to use. Additionally, there's an interesting trick for `<input type=checkbox>` inputs:

```
<input type=hidden name=example value=false>
<input type=checkbox name=example value=true>
```

In this way you can ensure if the checkbox is not checked, a value of `false` will be sent, and if it is, an array of `['false', 'true']` will be sent. The first will resolve to `False`, the second to `True`.

In [25]:
def boolean(input) -> bool:
	"""Convert the given input to a boolean value.
	
	Intelligently handles boolean and non-string values, returning as-is or passing to the bool builtin respectively.
	
	This process is case-insensitive.
	"""
	
	try:  # Eliminate unnecessary distractions and choice.
		input = input.strip().lower()
	except AttributeError:  # "Duck typing" — we aren't explicitly checking for "stringiness".
		return bool(input)  # Fall back to standard typecasting; handles bools, ints, &c.
	
	if input in ('yes', 'y', 'on', 'true', 't', '1'):
		return True
	
	if input in ('no', 'n', 'off', 'false', 'f', '0'):
		return False
	
	raise ValueError("Unable to convert {0!r} to a boolean value.".format(input))

In [26]:
boolean("yes")

True

In [27]:
boolean("f")

False

In [28]:
boolean(True)

True

In [29]:
boolean(False)

False

In [30]:
boolean(1)  # And so forth...

True

In [31]:
boolean(['false', 'true'])

True

In [32]:
import expectexception

In [33]:
%%expect_exception ValueError

boolean("hi")

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-33-dcb8e8559682> in <module>
----> 1 boolean("hi")

<ipython-input-25-f304bacb583a> in boolean(input)
     18                 return False
     19 
---> 20         raise ValueError("Unable to convert {0!r} to a boolean value.".format(input))

ValueError: Unable to convert 'hi' to a boolean value.
